In [ ]:
import os 
import sys

import torch


In [1]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from matplotlib.ticker import ScalarFormatter
import matplotlib.colors as mcolors

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps
from vecopsciml.operators import zero_order as zo
from vecopsciml.algebra import zero_order as azo

# Imports de las funciones creadas para este programa
from vecopsciml.operators.zero_order import Mx, My

In [2]:
from models import *

ModuleNotFoundError: No module named 'models'

In [9]:
project_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(project_root)

<module 'models.baseline' from '/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/models/baseline/models/baseline.py'>

In [33]:
project_root

'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'

In [10]:
from utils.folders import create_folder
from utils.load_data import load_data
from utils.checkpoints import load_results

In [11]:
DEVICE = torch.device("cpu")

In [12]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'

DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear.pkl')

BASELINE_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/baseline_model')
AUTOENCODER_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/autoencoder_model')
POD_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/POD_model')

In [13]:
sys.path.append(os.path.join(ROOT_PATH, r'models/baseline'))
sys.path.append(os.path.join(ROOT_PATH, r'models/autoencoder'))
sys.path.append(os.path.join(ROOT_PATH, r'models/POD'))
sys.path.append(os.path.join(ROOT_PATH, r'models/fourier'))

In [ ]:
# Load dataset
dataset = load_data(DATA_PATH)

# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [7]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [8]:
from models.baseline import BaselineNonlinearModel
from models import Autoencoder
from models import AutoencoderNonlinearModel
from models import PODNonlinearModel

Baseline model

In [9]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, 5, 10, 20]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

baseline_model = BaselineNonlinearModel(input_shape, predictive_layers, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(baseline_model.parameters(), lr=1e-2)
baseline_model, optimizer, lists = load_results(baseline_model, optimizer, BASELINE_RESULTS_PATH, map_location=DEVICE)

Autoencoder

In [ ]:
autoencoder_input_shape = y_train.values[0].shape
latent_space_dim = [15, 10, 3, 10, 15]
autoencoder_output_shape = y_train.values[0].shape

autoencoder = Autoencoder(autoencoder_input_shape, latent_space_dim, autoencoder_output_shape).to(DEVICE)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-2)

autoencoder, autoencoder_optimizer, autoencoder_lists = load_results(autoencoder, optimizer, BASELINE_RESULTS_PATH, map_location=DEVICE)